# Example of validation

Example of validation (statistics computation) on the ESA MAAP.

Load the libraries

In [ ]:
from osgeo import gdal
from gdalconst import GA_ReadOnly
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import LinearSegmentedColormap
import sys
sys.path.insert(0,'/projects/Scripts')
import roiStatistics

## 1. Definition of ROIS

List of considered Regions of Interest (ROI) and related in-situ measurements for La Lope

In [ ]:
rois = ['LOP01h', 'LOP02h', 'LOP03h', 'LOP07h', 'LOP08h', 'LOP09h', 'LOP10h', 'LOP11h', 'LOP12h', 'LPG01h', 'LPG02h']
measuredBiomasses = [0.0, 0.3, 15.5, 317.9, 290.4, 348.6, 375.1, 349.7, 321.4, 439.1, 547.3]
measuredHeights = [0.2, 0.3, 1.3, 33.2, 30.5, 39.7, 30.9, 28.6, 25.9, 31.8, 33.4]

## 2. Open and display product

Open biomass map

In [ ]:
biomassFilename = '/projects/biomass_GR.tiff'

input_image_driver = gdal.Open(biomassFilename, GA_ReadOnly)
estimatedBiomasses = input_image_driver.ReadAsArray()

Display biomass map

In [ ]:
plt.rcParams["figure.figsize"]=30,30
plt.rcParams.update({'font.size': 30})
fig, ax = plt.subplots()
customCmap = LinearSegmentedColormap.from_list('biomass', ['#F5F5DC', 'g', '#004500'])

# Display image
cax = ax.imshow(estimatedBiomasses, cmap=customCmap, extent=(0, input_image_driver.RasterXSize, 0, input_image_driver.RasterYSize))

# Set figure Layout:
ax.set_xlabel('Longitude (m)')
ax.set_ylabel('Latitude (m)')
cbar = fig.colorbar(cax)
cbar.set_label('Biomass (ton/ha)')

## 3. Compute statistics

Computation of the biomass image statistics over the ROIs

In [ ]:
shapeFolderPrefix = '/projects/s3-drive/catalog-data/Campaign_data/afrisar_dlr/afrisar_dlr_roi_'

stats = roiStatistics.getRoiStats(biomassFilename, shapeFolderPrefix, rois)
estimatedBiomasses = stats['mean']

Estimates per ROI

In [ ]:
i=0
print(rois[i])
print(stats['mean'][i])
print(stats['min'][i])
print(stats['max'][i])
print(stats['std'][i])
print(stats['count'][i])

Compute statistics

In [ ]:
statistics = roiStatistics.stats(estimatedBiomasses, measuredBiomasses)
[slope, intercept, r_value, p_value, std_err, bias, covariance, rmsd] = statistics
print('slope :' + str(slope))
print('intercept :' + str(intercept))
print('r_value :' + str(r_value))
print('p_value :' + str(p_value))
print('std_err :' + str(std_err))
print('bias :' + str(bias))
print('covariance :' + str(covariance))
print('rmsd :' + str(rmsd))

Trace estimated biomass vs in-situ measurements

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

# Trace of the estimated versus measured biomass according to the given parameters.
title = 'Title (' + '$r^2$=' + str(round(r_value**2, 2)) + ' | $RMSD$=' + str(round(rmsd, 1)) + ')'

# Trace values:
fig, ax = plt.subplots()
plt.title(title)
ax.plot(estimatedBiomasses, measuredBiomasses, 'bo')

# Trace reference line:
maximum = (max(max(estimatedBiomasses), max(measuredBiomasses), 199)//200 + 1) * 200
ax.plot([0,maximum], [0,maximum], '-k')

# Trace trend line
ax.plot([0, maximum], [intercept, maximum * slope + intercept], '-g')
blue_line = mlines.Line2D([], [], color='green', label='Trendline ($a$=' + str(round(slope, 2)) + ' | $b$=' + str(round(intercept, 2)) + ')')
ax.legend(handles=[blue_line], fancybox = True, shadow = True, loc = 'best', prop={'size':12})

# define axis:
ax.axis([0, maximum, 0, maximum])

# Set figure Layout:
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel('Estimated biomass (ton/ha)')
ax.set_ylabel('Ground truth biomass (ton/ha)')
ax.grid(True)

plt.show()